<a href="https://colab.research.google.com/github/Abbas-Muneer/Olympians/blob/DataScience/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2,preprocess_input
from tensorflow.keras.layers import Dense,Conv2D,GlobalAveragePooling2D,Input
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras import callbacks,optimizers
import numpy as np
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/model_training/


/content/drive/MyDrive/model_training


In [ ]:
for i in os.listdir("raw-img"):
  print(i)

ACNE
Skin Cancer
Psoriasis
Basal_Cell_Carcinoma
Melanocytic_Nevi
Atopic_Dermatitis
Melanoma
Eczema


In [ ]:
!ls raw-img/

 ACNE		     Basal_Cell_Carcinoma   Melanocytic_Nevi   Psoriasis
 Atopic_Dermatitis   Eczema		    Melanoma	      'Skin Cancer'


In [ ]:
for i in os.listdir("raw-img"):
  print(i,len(os.listdir("raw-img/" + i)))

ACNE 375
Skin Cancer 3543
Psoriasis 341
Basal_Cell_Carcinoma 1323
Melanocytic_Nevi 2020
Atopic_Dermatitis 481
Melanoma 1140
Eczema 18


In [ ]:
try:
  os.mkdir("train")
  os.mkdir("test")
except:
  pass  
for i in os.listdir("raw-img"):
  try:
    os.mkdir("train/" + i)
    os.mkdir("test/" + i)
  except:
    pass  
  for j in os.listdir("raw-img/" + i)[:1600]:
    os.rename("raw-img/"+i+"/"+j, "train/"+i+"/"+j)
  for j in os.listdir("raw-img/" + i)[:400]:
    os.rename("raw-img/"+i+"/"+j, "test/"+i+"/"+j)

In [12]:
for i in os.listdir("train"):
  print(i,len(os.listdir("train/" + i)))

ACNE 1600
Skin Cancer 1426
Psoriasis 1600
Basal_Cell_Carcinoma 1600
Melanocytic_Nevi 1600
Atopic_Dermatitis 1600
Melanoma 1600
Eczema 1600


In [ ]:
def img_Data(dir_path,target_size,batch,class_lst,preporcssing):
  if preporcssing:
    gen_object = ImageDataGenerator(preprocessing_function=preporcssing)
  else:
    gen_object = ImageDataGenerator()

  return(gen_object.flow_from_directory(dir_path,
                                        target_size=target_size,
                                        batch_size=batch,
                                        class_mode='sparse',
                                        classes = class_lst,
                                        shuffle=True))

In [ ]:
train_data_gen = img_Data("train",(224,224),500,os.listdir("train"),preprocess_input)
valid_data_gen = img_Data("test",(224,224),500,os.listdir("test"),preprocess_input)

Found 12799 images belonging to 8 classes.
Found 3200 images belonging to 8 classes.


In [ ]:
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2( input_shape=(224,224,3),
                                                             alpha=1.0,
                                                             include_top=False,
                                                             weights='imagenet',
                                                             input_tensor=None,
                                                             pooling=None,
                                                             classes=1000,
                                                             classifier_activation='softmax')
                                        
   


In [ ]:
base_model.trainable = False

In [ ]:
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024,activation='relu'))
model.add(Dense(8,activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
              

In [ ]:
elst = callbacks.EarlyStopping(monitor='val_loss',patience=5,mode='min')
save_ck = callbacks.ModelCheckpoint(' .mdl_wt.hdf5',save_best_only=True,monitor='val_loss',mode= 'min')

In [ ]:
model.fit(train_data_gen,batch_size=500,validation_data=valid_data_gen,callbacks=[elst,save_ck],epochs=10)

Epoch 1/10
26/26 [==============================] - ETA: 0s - loss: 1.2416 - accuracy: 0.5599  

/usr/local/lib/python3.8/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


26/26 [==============================] - 3463s 133s/step - loss: 1.2416 - accuracy: 0.5599 - val_loss: 0.9795 - val_accuracy: 0.5750
Epoch 2/10
26/26 [==============================] - 1053s 41s/step - loss: 0.7093 - accuracy: 0.7065 - val_loss: 0.8369 - val_accuracy: 0.6428
Epoch 3/10
26/26 [==============================] - 1065s 41s/step - loss: 0.6043 - accuracy: 0.7503 - val_loss: 0.7767 - val_accuracy: 0.6575
Epoch 4/10
26/26 [==============================] - 1085s 42s/step - loss: 0.5279 - accuracy: 0.7880 - val_loss: 0.7629 - val_accuracy: 0.6753
Epoch 5/10
26/26 [==============================] - 1081s 42s/step - loss: 0.4708 - accuracy: 0.8132 - val_loss: 0.8133 - val_accuracy: 0.6562
Epoch 6/10
26/26 [==============================] - 1086s 42s/step - loss: 0.4177 - accuracy: 0.8368 - val_loss: 0.7580 - val_accuracy: 0.6678
Epoch 7/10
26/26 [==============================] - 1065s 41s/step - loss: 0.3605 - accuracy: 0.8673 - val_loss: 0.7634 - val_accuracy: 0.6828
Epoch 8/1